<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="float:right; max-width: 250px; display: inline"  alt="Wikistat"/></a>
</center>

# [Scénarios d'Exploration Statistique](https://github.com/wikistat/Exploration)

# Introduction à l'[AFD](http://wikistat.fr/pdf/st-m-explo-afd.pdf) avec <a href="https://cran.r-project.org/"><img src="https://cran.r-project.org/Rlogo.svg" style="max-width: 40px; display: inline" alt="R"/></a>
# Exemples élémentaires
#### Résumé 
Illustration par un exemple jouet puis analyse de données socio-économiques par [ACP](http://wikistat.fr/pdf/st-m-explo-acp.pdf) et [Analyse Factorielle Discriminante](http://wikistat.fr/pdf/st-m-explo-afd.pdf) (AFD) avec R.

## 1 Introduction
Il y a différentes approches du problème de la discrimination selon l'objectif visé et donc la méthodologie mise en oeuvre. Nous nous limitons ici à l'approche descriptive ou factorielle (décomposition en facteurs). L'approche décisionnelle est traité en [saison 3](https://github.com/wikistat/Apprentissage). 

L'[Analyse Factorielle Discriminante](http://wikistat.fr/pdf/st-m-explo-afd.pdf) est un cas particulier d'[ACP](http://wikistat.fr/pdf/st-m-explo-acp.pdf) pour décrire la capacité de discrimination des $m$ classes d'une variable qualitative $Y$ par un ensemble de variables quantitatives $X^1,...,X^p$ observées sur $n$ individus.

**Q** Quelle est cette ACP?

**Q** Quel est le rang de ma matrice à diagonaliser et donc le nombre maximal de composantes d'une AFD?

Cette méthode encore appelée *canonical discriminant analysis* dans la littérature anglo-saxone est assez peu répandue et donc peu présente (procédure `CANDISC` de SAS/STAT), voire absente, des logiciels ou librairies. Son usage est néanmoins détaillé pour insister sur le rôle spécifique de la métrique de *Mahalabobis* qui se retrouve dans d'autres circonstances: analyse discriminante décisionnelle, $k$ plus proches voisins, détection d'anomalies.

Sachant que l'AFD est une ACP, il suffirait de dérouler en R les calculs matriciels décrit dans  la [vignette](http://wikistat.fr/pdf/st-m-explo-afd.pdf) pour aboutir au résultat. Néanmoins le choix est fait d'utiliser les ressources déjà présentes de la fonction d'anayse discriminante décisionnelle `lda` de R afin d'obtenir une exécution plus efficace.

Reda Boudjeltia de Bordeaux met à disposition une une [fonction R d'AFD](https://github.com/royceda/Data_Analysis) mais sans garantie de pérennité. 

## 2 Exemple jouet: les iris de Fisher
Il s'agit d'un jeu de données historique le plus utilisé pour illustrer toute méthode exploratoire, inférentielle ou prédictive. 
### 2.1 Description élémentaire
Les données sont déjà connues de R et décrivent 3 type d'iris par les longueurs et largeurs de leurs pétales et sépales.

In [ ]:
boxplot(iris[,1:4])

In [ ]:
summary(iris)

**Q** Quel est le graphique ci-dessous? Que dire de la séparation des classes?

In [ ]:
options(repr.plot.width=5, repr.plot.height=5)
pairs(iris[,1:4],col=iris$Species)

### 2.2 ACP des iris

In [ ]:
library(FactoMineR)
acp=PCA(iris[,1:4], graph=F)
options(repr.plot.width=4, repr.plot.height=4)
plot(acp,choix="var")

**Q** Interprétation des axes.

In [ ]:
plot(acp, choix="ind", habillage="ind",col.hab=iris$Species)

**Q** Quelle est la qualité de la représentation?

**Q** Que dire de la séparation naturelle des classes?

**Q** Que dire de la forme des nuages et donc de la structure de covariance intra-classe?

**Q** Coment caractériser la classe rouge?
### 2.3 AFD des iris

#### Avec la fonction R ad'hoc
L'option `type="GB"` détrermine les diviseurs de la variance et le choix de la métrique de Mahalanobis comme inverse le matrice de variance intraclasse.

In [ ]:
source("Data/AFD_procedures.r")
X=iris[,1:4]
y=as.factor(as.integer(iris[,5]))
resAFD = AFD(X,y,type="GB")

In [ ]:
plotAFD(resAFD)

**Q** Quel est la qualité de la représentation?

**Q** En tenant compte des échelles des axes, que devient la forme des nuages des classes?

**Q** Que représente le cosinus de l'angle entre un vecteur variable et un axe?

**Q** Les résultats ci-dessous sont obtenus avec la fonction ad'hoc d'AFD ou avec celle `lda`. Comparer.

In [ ]:
library(MASS)
# calcul de l'afd
resLDA=lda(iris[,1:4],iris$Species)

In [ ]:
resAFD$eig; resLDA$svd

**Q** Que sont les valeurs 150-3 et 3-1 ci-dessous permettant de passer des résultats d'une analyse à l'autre.

In [ ]:
(resLDA$svd**2)/(150-3)*(3-1)

In [ ]:
resLDA$scaling

In [ ]:
resAFD$U

**Q** Quelle est la dimension de l'espace de représentation?

Un graphique est aussi proposé mais incomplet:

In [ ]:
plot(resLDA, col=as.integer(iris$Species))

**Q** Que manque-t-il dans ce graphique?

**Q** Que dire de la forme des nuages en relation avec le choix de la métrique?

Calcul des coordonnées des barycentres des classes dans le nouveau repère.

In [ ]:
pred=predict(resLDA,data=iris[,1:4])
m=matrix(rep(0,6),nrow=3,ncol=2)
for (i in 1:3){
 for (j in 1:2){m[i,j]=mean(pred$x[unclass(iris$Species)==i,j])}}

In [ ]:
# graphe dans le premier plan
color=as.integer(iris$Species)
options(repr.plot.width=5, repr.plot.height=5)
plot(pred$x[,1],pred$x[,2],type="p",col=color,pch=19,cex=0.5,asp=1)
abline(0,0,h=0); abline(0,0,v=0)
text(m[,1],m[,2],labels=levels(iris$Species),cex=1,col=1:3)

**Q** Que dire de la forme des nuages autour des barycentres? Remarquer le rôle important du paramètre `asp=1`.

**Q** Que dire des capacités de discrimination des variables?

Il manque encore une représentation des variables afin de mettre en évidence celles les plus liées aux axes et donc les plus discriminantes.

## 3 Données socio-économiques départementales *vs.* régionales
Cette section se focalise sur l'exploration de données socio-économiques observées sur les différents départements regroupés en régions administratives. La question posée est: les régions définies géographiquement sont elles relativement homogènes sur ces aspects socio-économiques. Ce peut être une analyse éclairante de la volonté de supprimer les départements et regrouper des régions. On répond à cette question de façon indirecte et exploratoire, en cherchant à savoir si les variables socio-économiques permettent de discriminer les régions.


### 3.1 Les données
Les données proviennent du Groupe d'Étude et de Réflexion Inter-régional (GERI). Elles décrivent, quatre grands thèmes: la démographie, l'emploi, la fiscalité directe locale, la criminalité, de chacun des départements français métropolitains et la Corse. Les indicateurs ont été observés pendant l'année 1990, ils sont, pour la plupart, des taux calculés relativement à la population totale du département concerné. Voici leur liste~:
- urbr: indicateur de concentration de la population mesurant le caractère urbain ou rural d''un département,
- txcr: taux de croissance de la population sur la période intercensitaire 1982-1990,
- jeun: part des 0-19 ans dans la population totale, 
- age: part des plus de 65 ans dans la population totale,
- fe90: taux de fécondité (pour 1000) égal au nombre de naissances rapportés au nombre de femmes fécondes (15 à 49 ans) en moyenne triennale, 
- etra: part des étrangers dans la population totale, 
- chom: taux de chômage, 
- crim: taux de criminalité~: nombre de délits par habitant,
- fisc: produit, en francs constants 1990 et par habitant des quatre taxes directes locales (professionnelle, habitation, foncier bâti, foncier non bâti).

Parts de chaque profession et catégorie socioprofessionnelle (PCS) dans la population active occupée du département~:
- agri: agriculteurs,
- arti: artisans, 
- cadr: cadres supérieurs, 
- empl: employés, 
- ouvr: ouvriers, 
- prof: professions intermédiaires,

En plus de ces variables, la première colonne identifie le département, la deuxième sa région avant le regroupement de 2016, la dernière une proposition de regroupement géographique en 6 grandes régions différentes de celles choisies en 2016.

Une première étape descriptive n'a pas conduit à des re-transformations des variables. 

**Q** Celles-ci sont, pour la plupart, déjà des taux, pourquoi?. 

**Q** dan le même ordre d'idée, les départements d'Ile de France sont exclus. Pourquoi?

Une ACP permet de se faire une première idée sur l'organisation de ces données. 

## 3.2 Analyse en composantes principales

In [ ]:
depart=read.csv("Data/depart.csv",row.names=1)
summary(depart)

In [ ]:
acp=PCA(depart[,3:17],graph=F,ncp=10)
options(repr.plot.width=4, repr.plot.height=3)
boxplot(acp$ind$coord)

**Q** Quelle dimension retenir?

In [ ]:
options(repr.plot.width=5, repr.plot.height=5)
plot(acp,choix="var")

**Q** Interprétation des axes?

In [ ]:
plot(acp, choix="ind", habillage="ind",col.hab=depart$groupreg)

**Q** Homogénéité des classes?

## 3.3 Analyse factorielle discriminante
On se propose de mettre en évidence les plus grandes disparités inter-régionales et donc de rechercher les variables ou combinaisons de variables expliquant au mieux le découpage régional. Autre question: les régions administratives sont-elles homogènes d'un point de vue socio-économique. Pour simplifier, nous procédons à des regroupements afin de construire des régions moins nombreuses comprenant des nombres de départements plus semblables. La région Ile de France, trop particulière et donc trop facile à discriminer, a été laissée à part. Les regroupements proposés sont différents de ceux finalement adoptés en 2016 mais l'objectif est finalement assez similaire.

In [ ]:
library(MASS)
# calcul de l'afd
dep.afd=lda(depart[,3:17],depart$groupreg)
print(dep.afd)

Comme précédemmnent, le graphique inclus est peu adapté, il est préférable de construire à la main le plan factoriel

In [ ]:
# calcul des coordonnées barycentres
dep.pred=predict(dep.afd,data=depart[,3:17])
m=matrix(rep(0,16),nrow=8,ncol=2)
for (i in 1:8){
 for (j in 1:2){
 m[i,j]=mean(dep.pred$x[unclass(depart$groupreg)==i,j])}}

In [ ]:
# graphe dans les axes 1 et 2
color=as.integer(depart$groupreg)
plot(dep.pred$x[,1],dep.pred$x[,2],cex=0.001)
text(dep.pred$x[,1],dep.pred$x[,2],labels=row.names(depart),col=color)
abline(0,0,h=0); abline(0,0,v=0)
text(m[,1],m[,2],labels=levels(depart$groupreg),cex=1,col=1:8)

**Q** Quelles sont les régions géographiques économiquement homogènes? Proches? 

**Q** Quelle est la région la plus disparate.